In [ ]:
from google.colab import drive
drive.mount('/content/drive/')  # Получить доступ к диску в колабе
import os
import cv2

Mounted at /content/drive/


In [ ]:
# Программа вычисления суммы площадей полигонов по классам в пределах выбранной
# папки. Площадь суммы полигонов приведена к размерам кадра 640 х 640 пикселей.
# Arc Аркадий Соловьев
# Для определения размера полигона необходимо указать путь к папке
# annotation_folder с файлами аннотации.
# Для определения текущего размера  кадра необходимо указать путь к любому файлу .jpg,
# для которого есть аннотация в папке annotation_folder.

def calculate_polygon_area(vertices):  # Функция вычисления площади полигонов
    n = len(vertices)                  # Число вершин полигона
    if n < 3:    # Если число верши полигона меньше 3 его площадь не вычисляется
        return 0

    area = 0     # Площадь полигона
    j = n - 1

    for i in range(n):                 # Вычисление площади полигона
        x1, y1 = vertices[i]
        x2, y2 = vertices[j]
        area += (x1 + x2) * (y2 - y1)
        j = i

    return abs(area) / 2.0 # Функция возвращает площадь полигона

# Инициализируем словарь для хранения суммарных площадей по классам
class_areas = {}

# Инициализируем словарь для хранения количества размеченных объектов (полигонов) по классам
class_number = {}

# Папка с файлами аннотации
annotation_folder = "/content/drive/MyDrive/1 Датасет 4.0/far/train/labels"  # Указать путь к папке с файлами аннотации

# Указать путь к любому файлу для которого есть аннотация в папке annotation_folder:
image_file = "/content/drive/MyDrive/1 Датасет 4.0/Close/train/images/DJI_0316_frame_1230_0.png"

# Определить размеры изображения, для которого есть файл аннотации
img = cv2.imread(image_file)       # Прочитать файл изображения
h, w, c = img.shape                # h - высота, w - ширина, c - число каналов цвета

factor_size = h * w/ 640 ** 2      # Поправочный коэффициент, учитывает отличие размера файла от 640 х 640

# Шаг 1: Пройдем по всем файлам аннотации в папке
for filename in os.listdir(annotation_folder):
    if filename.endswith(".txt"):
        # Получаем полный путь к файлу аннотации
        annotation_file = os.path.join(annotation_folder, filename)

        # Шаг 2: Прочитаем файл аннотации
        with open(annotation_file, "r") as file:
            annotation_data = file.read().strip()

        # Шаг 3: Разберем информацию о полигоне и классе объекта
        lines = annotation_data.split("\n")   # Строки в файле аннотации
        for line in lines:
            parts = line.split()  # Разделение строки на части по пробелам
            if len(parts) >= 5:
                class_name = parts[0]  # Идентификатор имени класса (первое число в строке)
                polygon_data = [float(part) for part in parts[1:]]  # Преобразуем строки в числа
                # Список пар x, y координат вершин полигона
                x_y_pairs = [(polygon_data[i], polygon_data[i+1]) for i in range(0, len(polygon_data), 2)]

                # Шаг 4: Вычислим площадь полигона
                area = calculate_polygon_area(x_y_pairs)

                # Шаг 5: Обновим суммарную площадь и количество элементов (полигонов) для данного класса
                if class_name in class_areas:  # Если класс с именем class_name существует
                    class_areas[class_name] += area  # Увеличить его площадь
                else:
                    class_areas[class_name] = area   # Создать новый класс

                if class_name in class_number:  # Если класс с именем class_name существует
                    class_number[class_name] += 1  # Увеличить число элементов класса
                else:
                    class_number[class_name] = 1   # Число элементов в классе равно 1

# Шаг 6: Выведем суммарное количество полигонов и суммарные площади полигонов для каждого класса
print(' Суммарная площадь полигонов (в условных единицах) приведена к размеру кадра 640 х 640 пикселей (коэффициент factor_size)')
for i in class_number.keys():
    print(f" Класс объекта: {i} количество полигонов = {class_number[i]:>3}, суммарная площадь полигонов = {class_areas[i] * factor_size:>7.3f}")


 Суммарная площадь полигонов (в условных единицах) приведена к размеру кадра 640 х 640 пикселей (коэффициент factor_size)
 Класс объекта: 0 количество полигонов = 16336, суммарная площадь полигонов = 557.076
 Класс объекта: 1 количество полигонов = 7751, суммарная площадь полигонов = 326.597
